In [1]:
import numpy as np

In [15]:
def create_markov_chain():
    markov_chain = {}
    for from_base in 'ATGC':
        #generate slice
        slice_points = sorted([0] + [1] + [np.random.random() for i in range(3)])
        #from slice to prob
        transition_prob = [slice_points[i+1] - slice_points[i] 
                           for i in range(len(slice_points)-1)]
        markov_chain[from_base] = {base:p 
                                   for base,p in zip('ATGC', transition_prob)}
    return markov_chain

mc = create_markov_chain()
mc

{'A': {'A': 0.016071984070559542,
  'C': 0.5790220168565988,
  'G': 0.14749309923907983,
  'T': 0.2574128998337618},
 'C': {'A': 0.5266966964970067,
  'C': 0.08403246305106093,
  'G': 0.29523226462261043,
  'T': 0.09403857582932196},
 'G': {'A': 0.18323059156228694,
  'C': 0.25332755530458084,
  'G': 0.15782709599801636,
  'T': 0.40561475713511586},
 'T': {'A': 0.5319798878305895,
  'C': 0.07813237789709893,
  'G': 0.19713456062583534,
  'T': 0.19275317364647626}}

In [16]:
def check_transition_probabilities(markov_chain):
    
    for from_base in markov_chain:
        s = sum([markov_chain[from_base][to_base]
                for to_base in markov_chain[from_base]])
        if abs(s-1) > 1E-15:
            raise ValueError('Wrong sum: {} for ”{}"'.format(s, from_base))
check_transition_probabilities(mc)

In [18]:
def mutate_via_markov_chain(dna, markov_chain):
    dna_list = list(dna)
    mutation_site = np.random.randint(0, len(dna_list))
    from_base = dna[mutation_site]
    to_base = draw(markov_chain[from_base])
    dna_list[mutation_site] = to_base
    return ''.join(dna_list)

def draw(prob_dist):
    tot = 0
    r = np.random.random()
    for to_base in prob_dist:
        tot += prob_dist[to_base]
        if r < tot:
            return to_base


In [19]:
dna = "ATCGAGCAGCGCAATTACGAGCATCAGACGAGATCCCTTTATGACTATT"
print('Starting DNA: {}'.format(dna))

mc = create_markov_chain()
print('Transition probs:\n{}'.format(mc))
nmutations =10000
for i in range(nmutations):
    dna=mutate_via_markov_chain(dna, mc)
    
print('DNA after {} mutations: {}'.format(nmutations, dna))

Starting DNA: ATCGAGCAGCGCAATTACGAGCATCAGACGAGATCCCTTTATGACTATT
Transition probs:
{'A': {'A': 0.015489400027195677, 'T': 0.10882063363096917, 'G': 0.7962729865440858, 'C': 0.07941697979774931}, 'T': {'A': 0.03077984091478614, 'T': 0.4722456051405861, 'G': 0.3051495927775103, 'C': 0.19182496116711745}, 'G': {'A': 0.22295394189781115, 'T': 0.05595678185843789, 'G': 0.7060084717140624, 'C': 0.015080804529688518}, 'C': {'A': 0.5329190347449304, 'T': 0.13530077258640738, 'G': 0.21987750753763902, 'C': 0.1119026851310232}}
DNA after 10000 mutations: GTGGGTAGGGGGCGAAAGGAATGAGGTGGGTGGATAGAGGGGTGGGTGG


In [26]:
def check_draw_approx(discrete_probdist, N=1000000):
    frequencies = {value: 0 for value in discrete_probdist}
    for i in range(N):
        value = draw(discrete_probdist)
        frequencies[value] += 1
    for value in frequencies:
        frequencies[value] /= float(N)
    print( "\n".join(['{}: {:.4f} (exact {:.4f})'.format(v, frequencies[v], discrete_probdist[v])
           for v in frequencies]))
check_draw_approx(mc['A'])

A: 0.0158 (exact 0.0155)
T: 0.1085 (exact 0.1088)
G: 0.7962 (exact 0.7963)
C: 0.0795 (exact 0.0794)


# Ex -- compute the probability of P(Y=b) where b is any nucleotide

# P(Y=b)=P(X=A∪Y=b)+P(X=C∪Y=b)+P(X=G∪Y=b)+P(X=T∪Y=b)

## P(X=A∪Y=b) = P(X=A) P(Y=b  | X=A)      